In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('tweets_translated.csv')
df = df.drop(df.columns[0], axis=1)

In [ ]:
df.isnull().sum(axis=0)

id                     0
conversation_id        0
created_at             0
date                   0
time                   0
timezone               0
user_id                0
username               0
name                   1
place              56541
tweet                  0
language               0
mentions               0
urls                   0
photos                 0
replies_count          0
retweets_count         0
likes_count            0
hashtags               0
cashtags               0
link                   0
retweet                0
quote_url          54772
video                  0
thumbnail          46426
near               56566
geo                56566
source             56566
user_rt_id         56566
user_rt            56566
retweet_id         56566
reply_to               0
retweet_date       56566
translate          56566
trans_src          56566
trans_dest         56566
time_stamp             0
Unnamed: 0.1        4455
Unnamed: 0          4455
dtype: int64

In [ ]:
#Droping all coulumns that are empty or with many missing values.
df = df.drop(['retweet_date', 'translate', 'trans_src', 'trans_dest', 'retweet_id', 
             'user_rt', 'user_rt_id', 'geo', 'near', 'place', 'source', 'quote_url', 'timezone', 'conversation_id', 'created_at'], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56566 entries, 0 to 56565
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              56566 non-null  int64  
 1   date            56566 non-null  object 
 2   time            56566 non-null  object 
 3   user_id         56566 non-null  int64  
 4   username        56566 non-null  object 
 5   name            56565 non-null  object 
 6   tweet           56566 non-null  object 
 7   language        56566 non-null  object 
 8   mentions        56566 non-null  object 
 9   urls            56566 non-null  object 
 10  photos          56566 non-null  object 
 11  replies_count   56566 non-null  int64  
 12  retweets_count  56566 non-null  int64  
 13  likes_count     56566 non-null  int64  
 14  hashtags        56566 non-null  object 
 15  cashtags        56566 non-null  object 
 16  link            56566 non-null  object 
 17  retweet         56566 non-null 

In [ ]:
df = df.drop_duplicates(subset='id')

In [ ]:
df.shape

(55692, 24)

In [ ]:
len(df['id'].unique())

55692

## Cleaning Tweets 

In [ ]:
def hashtag_removal(r):
    """
    function to remove hashtags from a tweet
    """
    twt=r['tweet'].lower()
    tags=list(r['hashtags'][1:-1].split(','))
    for i in tags:
        i=i.strip()
        i=i[1:-1]
        twt=twt.replace('#'+i,'')
    return twt 
df['clean_tweet']=df.apply(lambda x:hashtag_removal(x),axis=1)

In [ ]:
!pip install tweet-preprocessor
import preprocessor as p
#Clening the tweets using tweet-preprocesor
df['clean_tweet']=df['clean_tweet'].apply(lambda x:p.clean(x))

  Using cached tweet_preprocessor-0.6.0-py3-none-any.whl (27 kB)
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/l

In [ ]:
import regex as re
def punctuation_removal(r):
    """
    Removing punctioion from tweet
    """
    ls=re.findall(r'&(\w+);', r)
    for i in ls:
        r=r.replace(f'&{i};','')
    punc = '’’!()-[]{};:’”\,<>./?@#$%^&*_~’’' 
    for ele in r:
         if ele in punc:
            r = r.replace(ele, '')
         return r
df['clean_tweet']=df['clean_tweet'].apply(lambda x:punctuation_removal(x))

In [ ]:
#Remove Digits
df['clean_tweet'] = df['clean_tweet'].astype(str).str.replace(' \d+', '')
df['clean_tweet']

0        not saying your wrong about the safety car thi...
1        ferrari threw away a possible podium, indecisi...
2        ricciardo: it's only ninth but still important...
3                                                kg equip?
4        i note the very ominous and mysterious "didn't...
                               ...                        
56561             tesla should compete in formula electric
56562            what every ferrari f1 driver is scared of
56563      grande charles, grande carlos, really good work
56564    pierre said the only french food yuki hasnt tr...
56565    you're not going to argue because there's no f...
Name: clean_tweet, Length: 55692, dtype: object

## Tokenization

In [ ]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
 return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
df['tokenized_tweet'] = df['clean_tweet'].apply(lambda x:lemmatize_text(x))
df['tokenized_tweet']

[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/repl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0        [not, saying, your, wrong, about, the, safety,...
1        [ferrari, threw, away, a, possible, podium, ,,...
2        [ricciardo, :, it's, only, ninth, but, still, ...
3                                           [kg, equip, ?]
4        [i, note, the, very, ominous, and, mysterious,...
                               ...                        
56561      [tesla, should, compete, in, formula, electric]
56562    [what, every, ferrari, f1, driver, is, scared,...
56563    [grande, charles, ,, grande, carlos, ,, really...
56564    [pierre, said, the, only, french, food, yuki, ...
56565    [you're, not, going, to, argue, because, there...
Name: tokenized_tweet, Length: 55692, dtype: object

**Stop Words removal**

In [ ]:
stop_words = set(stopwords.words('english'))
df['tokenized_tweet'] = df['tokenized_tweet'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
df.to_csv('cleand_tweets.csv', index=False)